In [1]:
import sys
import asyncio
import logging

import numpy as np

from lsst.ts import salobj


from lsst.ts.standardscripts.auxtel.attcs import ATTCS

Before proceeding, add a logger and set logging level to `DEBUG` so we can get status message from the various tasks.

In [2]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

Next we initialize two high level classes to command the ATTCS and LATISS.

In [3]:
attcs = ATTCS()

In [4]:
await attcs.start_task

Read historical data in 0.07 sec
Read historical data in 0.76 sec
Read historical data in 1.22 sec
Read historical data in 1.77 sec
Read historical data in 2.17 sec
Read historical data in 2.73 sec
Read historical data in 3.68 sec
RemoteEvent(ATDome, 0, logMessage) falling behind; read 20 messages


[None, None, None, None, None, None, None]

RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 12 messages


In [10]:
await attcs.atdome.evt_heartbeat.next(flush=True)

In [21]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "current",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

Gathering settings.
Received settings from users.: {'ataos': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'current', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Settings versions: {'ataos': 'current', 'atmcs': '', 'atptg': '', 'atpneumatics': '', 'athexapod': 'current', 'atdome': 'test.yaml', 'atdometrajectory': ''}
Enabling all components
Enabling  atmcs
Enabling  atptg
Enabling  ataos
Enabling  atpneumatics
Enabling  athexapod
Enabling  atdome
Enabling  atdometrajectory
[atmcs]::[<State.ENABLED: 2>]
[atptg]::[<State.ENABLED: 2>]
[ataos]::[<State.ENABLED: 2>]
[atpneumatics]::[<State.ENABLED: 2>]
[athexapod]::[<State.ENABLED: 2>]
[atdome]::[<State.ENABLED: 2>]
[atdometrajectory]::[<State.ENABLED: 2>]
All components enabled.


In [14]:
await attcs.atdome.cmd_moveShutterMainDoor.set_start(
                open=True,
                timeout=attcs.open_dome_shutter_time)

AckTimeoutError: msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=1185069293, ack=<SalRetCode.CMD_INPROGRESS: 301>, error=0, result='')

In [ ]:
await attcs.atdome.cmd_homeAzimuth.start()

In [ ]:
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth=20)

In [ ]:
attcs.dome_flat_az=20
attcs.tel_flat_az=205.7
attcs.tel_flat_el=39.

In [ ]:
await attcs.prepare_for_flatfield()

In [ ]:
await attcs.atptg.cmd_azElTarget.set_start(azDegs=205.7, elDegs=39.)

In [ ]:
await attcs.atptg.cmd_stopTracking.start()

In [15]:
await attcs.atpneumatics.cmd_openM1Cover.start()

In [16]:
await attcs.ataos.cmd_enableCorrection.set_start(m1=True, timeout=attcs.long_timeout)

In [ ]:
await attcs.atpneumatics.cmd_closeM1Cover.start()

In [22]:
await attcs.atmcs.cmd_setInstrumentPort.set_start(port=0)

RemoteEvent(ATMCS, 0, target) falling behind; read 91 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 27 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 39 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 23 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 60 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 27 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 91 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 16 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 18 messages


In [ ]:
await attcs.atdometrajectory.cmd_enable.start()

In [18]:
await attcs.atptg.cmd_azElTarget.set_start(azDegs=45, elDegs=79.)

In [ ]:
await attcs.at

In [ ]:
await attcs.atdome.cmd_moveAzimuth.set_start(azimuth=90)

In [20]:
await attcs.atmcs.cmd_stopTracking.start()

In [ ]:
await attcs.startup()

In [ ]:
await attcs.atpneumatics.cmd_closeInstrumentAirValve.start()

In [ ]:
await attcs.atpneumatics.cmd_closeMasterAirSupply.start()

In [ ]:
await attcs.ataos.cmd_enableCorrection.set_start(m1=False, hexapod=True, timeout=30)

In [ ]:
await attcs.ataos.cmd_disableCorrection.set_start(m1=False, hexapod=True, timeout=30)

In [ ]:
await attcs.atpneumatics.cmd_openM1Cover.start()

In [ ]:
await attcs.atdome.cmd_homeAzimuth.start()

Now we can use LATISS high level method `take_bias` to take a series of bias images.

In [ ]:
await attcs.startup()

In [ ]:
attcs.check.atdome = False

In [ ]:
await attcs.atdome.cmd_closeShutter.start()

Select instrument port. This is not done at the startup.

In [ ]:
time = await attcs.atptg.tel_timeAndDate.aget()

In [ ]:
time.lst

In [ ]:
await attcs.slew_icrs(ra=time.lst, dec=-50., slew_timeout=120)

In [ ]:
await attcs.athexapod.cmd_moveToPosition.set_start()

In [ ]:
await salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY)

In [ ]:
await asyncio.sleep(5)
await asyncio.gather(salobj.set_summary_state(attcs.atmcs, salobj.State.STANDBY, timeout=120),
                     salobj.set_summary_state(attcs.atptg, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.ataos, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.atpneumatics, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.athexapod, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.atdome, salobj.State.STANDBY, timeout=30),
                     salobj.set_summary_state(attcs.atdometrajectory, salobj.State.STANDBY, timeout=30),
                    return_exceptions=True)

In [32]:
await salobj.set_summary_state(attcs.atptg, salobj.State.ENABLED, timeout=30)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

RemoteEvent(ATMCS, 0, target) falling behind; read 79 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 32 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 15 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, measuredMotorVelocity) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 46 messages


In [26]:
await attcs.slew_icrs(ra="03 58 01.7669489", dec="-13 30 30.669770", target_name="gam-Eri")

Flushing events
Scheduling check coroutines
Sending command
process as completed...


TimeoutError: 

Got True
Telescope in position.


In [ ]:
HD 20894

In [30]:
attcs.check.atdome = False
await attcs.slew_icrs(ra="03 21 24.0043600176", dec="-23 38 06.552850356", rot_pa=120., target_name="HD20894")

Flushing events
Scheduling check coroutines
Sending command
process as completed...
Got False
Telescope not in position


TimeoutError: 

Got True
Telescope in position.
athexapod not in <State.ENABLED: 2>: <State.DISABLED: 1>
RemoteEvent(ATMCS, 0, target) falling behind; read 68 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 26 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 14 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 17 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 17 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 17 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 18 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 18 messages
Remote

In [29]:
attcs.slew_icrs?

Signature:
attcs.slew_icrs(
    ra,
    dec,
    rot_sky=None,
    rot_pa=0.0,
    target_name='slew_icrs',
    slew_timeout=240.0,
    track_id=0,
)
Docstring:
Slew the telescope and start tracking an Ra/Dec target in ICRS
coordinate frame.

Parameters
----------
ra : `float` or `str`
    Target RA (hour).
dec : `float` or `str`
    Target Dec (deg).
rot_sky : `float` or `str`
    Target sky position angle (deg). Default is `None`, which means
    use `rot_pa`.
rot_pa : `float` or `str`
    Target rotator position angle (deg). Ignored if `rot_sky` is
    given (Default = 0).
target_name :  `str`
    Target name.
slew_timeout : `float`
    Timeout for the slew command (second).
track_id : `int`
    Identifier for the target. This number will be attacked to all
    the commands sent by the pointing component.
File:      ~/repos/ts_standardscripts/python/lsst/ts/standardscripts/auxtel/attcs.py
Type:      method


In [24]:
await attcs.atptg.cmd_stopTracking.start()

atptg not in <State.ENABLED: 2>: <State.FAULT: 3>
